In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as ticker
import re
from scipy import stats as st
from scipy.stats import norm, beta

In [ ]:
%run ./EDA1V_new.ipynb
;

## Continuous numeric features

In [ ]:
# Numeric variable .description() check
# 
# min/max vs quantiles, mean, and standard deviation – looking for outliers/sensible results
# 
# Distribution plot examination
# 
# can use .hist() command or a more customized subplot group or for-each loop to plot a seperate histogram of each variable

In [ ]:
dfXY.head()

In [ ]:
#Numeric vars
numerical_var_series=dfXY[dfXY.columns[dfXY.dtypes == np.float64]].columns#.to_list()#[0:5]
print(type(numerical_var_series))
print(len(numerical_var_series))
numerical_var_series

In [ ]:
#Create the different groups of columns to plot
proceso_vars_series=dfXY[numerical_var_series[(~numerical_var_series.str.contains("AP1_Furnace",regex=False)) & (~numerical_var_series.str.contains("pyro2",regex=False))]].columns#.head(1)
zoneTemp_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Temp.*",regex=True)]].columns#.head(1)

zoneAirGas_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*Zone.*Air.*|.*Zone.*Gas.*",regex=True)]].columns#.head(1)

recuperator_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains("AP1_Furnace.*Dilution|Recuperator|RWP|Combustion.*",regex=True)]].columns#.head(1)

pyro_vars_series=dfXY[numerical_var_series[numerical_var_series.str.contains(".*pyro.*|.*Pyro.*",regex=True)]].columns#.head(1)

rest_vars_series=dfXY[[var for var in numerical_var_series.to_list() if var not in proceso_vars_series.to_list() and var not in zoneTemp_vars_series.to_list() and var not in zoneAirGas_vars_series.to_list() and var not in recuperator_vars_series.to_list() and var not in pyro_vars_series.to_list()]].columns#.head(1)

In [ ]:
#Plot histograms
#dfXY[dfXY.columns[dfXY.dtypes == np.float64]].hist(figsize=(20,10))#Too many vars: 175
print("There are {} numerical vars".format(dfXY[dfXY.columns[dfXY.dtypes == np.float64]].shape))
print("There are {} numerical vars of tbl proceso".format(len(proceso_vars_series)))
print("There are {} numerical vars of zone temp".format(len(zoneTemp_vars_series)))
print("There are {} numerical vars of zone air/gas flow".format(len(zoneAirGas_vars_series)))
print("There are {} numerical vars of recuperator".format(len(recuperator_vars_series)))
print("There are {} numerical vars of pyro".format(len(pyro_vars_series)))
print("The rest of numerical vars of furnace are {} ".format(len(rest_vars_series)))

if dfXY[dfXY.columns[dfXY.dtypes == np.float64]].shape[1]==len(proceso_vars_series)+len(zoneTemp_vars_series)+len(zoneAirGas_vars_series)+len(recuperator_vars_series)+len(pyro_vars_series)+len(rest_vars_series):
    print("All groups sum total length")

### Plot proceso vars: 4

In [ ]:
#Plot proceso boxplot: 4 vars

fig,axes = plt.subplots(4,1,figsize=(20,7))
sns.boxplot(data=dfXY[proceso_vars_series[0]],orient='h',ax=axes[0],width=0.2)#,width=0.2
axes[0].set_xlabel(xlabel=proceso_vars_series[0],labelpad=10,fontsize="x-large")
axes[0].tick_params(axis='x', labelsize=15)
sns.boxplot(data=dfXY[proceso_vars_series[1]],orient='h',ax=axes[1],width=0.2)#,width=0.2
axes[1].set_xlabel(xlabel=proceso_vars_series[1],labelpad=10,fontsize="x-large")
axes[1].tick_params(axis='x', labelsize=15)
sns.boxplot(data=dfXY[proceso_vars_series[2]],orient='h',ax=axes[2],width=0.2)#,width=0.2
axes[2].set_xlabel(xlabel=proceso_vars_series[2],labelpad=10,fontsize="x-large")
axes[2].tick_params(axis='x', labelsize=15)
sns.boxplot(data=dfXY[proceso_vars_series[3]],orient='h',ax=axes[3],width=0.2)#,width=0.2
axes[3].set_xlabel(xlabel=proceso_vars_series[3],labelpad=10,fontsize="x-large")
axes[3].tick_params(axis='x', labelsize=15)
fig.suptitle("Variables from Proceso boxplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper')
fig.tight_layout(pad=3)

In [ ]:
#Define func hist plot rest vars
def func_hist_plot_rest(vars_series:pd.Series,title:str,ncol:int,hue:str=None,colour:str="olive"):
    ncol=ncol
    nrow=dfXY[vars_series].shape[1]//ncol+1
    var=0
    
    fig,axes = plt.subplots(nrow,ncol,figsize=(20,nrow*20/5))
    fig.suptitle(f"Distribution of numerical variables from {title}",fontsize=30)
    
    for r in range(nrow):
        for c in range(ncol):
            if var<len(vars_series.to_list()):
                if re.search(r'.*Total.*',vars_series[var]):
                    sns.histplot(data=dfXY,x=vars_series[var],ax=axes[r,c],kde=True,color=colour,hue=hue)
                else:
                    if len(dfXY[vars_series[var]].unique())>1:
                        sns.histplot(data=dfXY,x=vars_series[var],ax=axes[r,c],kde=True,color=colour,hue=hue)
                    else:
                        sns.histplot(data=dfXY,x=vars_series[var],ax=axes[r,c],color=colour,hue=hue)
                axes[r,c].set_title(label=vars_series[var],loc='center',pad=10,fontsize="x-large")
                axes[r,c].set_xlabel(xlabel="",labelpad=10)
                axes[r,c].set_ylabel(ylabel="",labelpad=15)
                axes[r,c].axvline(tbl_desc.loc['cut-',vars_series[var]], color="red", dashes=(6, 2)) #Cut-
                axes[r,c].axvline(tbl_desc.loc['cut+',vars_series[var]], color="red", dashes=(6, 2)) #Cut+
                axes[r,c].tick_params(axis='x', labelsize=15)
                axes[r,c].tick_params(axis='y', labelsize=15)
                var+=1
                if c==0:
                    axes[r,c].set_ylabel(ylabel="Counts",labelpad=15,fontsize="large")
            #remove unused axes
            #This method removes subplots with hist of vars that have a unique value
            # if not axes[r,c].lines: #ax.lines is a matplotlib.axes._base._AxesBase.ArtistList,tells u number of lines in the subplot, if 0 then is empty
            #     axes[r,c].set_visible(False)
            #print(len(axes[r,c].get_children()))
            if len(axes[r,c].get_children()) <=10:#empty plots have 10 children
                axes[r,c].set_visible(False)
    
    
    # for ax in axes.flatten(): #axes.flatten() is a np.array of matplotlib.axes._axes.Axes
    #     if not ax.lines: #ax.lines is a matplotlib.axes._base._AxesBase.ArtistList,tells u number of lines in the subplot, if 0 then is empty
    #         ax.set_visible(False)
    
    #Keep outer labels
    # for ax in fig.get_axes():
    #     ax.label_outer()
    
    #Make space between subplots
    fig.tight_layout(pad=3)

In [ ]:
#Plot proceso: 4 vars
func_hist_plot_rest(vars_series=proceso_vars_series,title="proceso",ncol=2,hue=None,colour="olive")
#NOTES: CurrentWidth and NetWeight bimodal maybe bc widthID

### Plot zone temp: 20 vars

In [ ]:
#Plot bosplot zone temp: 20 vars

fig,axes = plt.subplots(figsize=(15,3))
sns.boxplot(data=dfXY[zoneTemp_vars_series[zoneTemp_vars_series.str.contains("CV",regex=False)]],orient='h',ax=axes,width=0.1)
axes.set_xlabel(xlabel=zoneTemp_vars_series[zoneTemp_vars_series.str.contains("CV",regex=False)][0],labelpad=10,fontsize="large")
axes.tick_params(axis='x', labelsize=15)
fig.suptitle("Zone Temp Zone0 CV boxplot",fontsize=20,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper')
#Remove ylabel
#axes.set_ylabel(ylabel='')
#plt.setp(axes, ylabel="")
#axes.set(ylabel=None)
#plt.yticks("")
axes.yaxis.set_major_locator(ticker.NullLocator())

In [ ]:
#Plot bosplot zone temp: 20 vars
fig,axes = plt.subplots(figsize=(20,15))
sns.boxplot(data=dfXY[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]],orient='h',ax=axes)
axes.tick_params(axis='x', labelsize=15)
axes.tick_params(axis='y', labelsize=15)
fig.suptitle("Zone Temp variables boxplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper', rc={"axes.labelsize":10})
#sns.set(font_scale=1)
fig.tight_layout(pad=3)

In [ ]:
#Plot violinplot zone temp: 20 vars
fig,axes = plt.subplots(figsize=(20,20))
sns.violinplot(data=pd.melt(dfXY[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)].tolist()+["SteelFamCluster"]].reset_index(),id_vars=["ts","SteelFamCluster"]),orient='h',ax=axes,x="value",y="variable",hue="SteelFamCluster",split=True,cut=0,inner_kws=dict(box_width=6, whis_width=2),density_norm="count")#,hue="SteelFamCluster"
axes.tick_params(axis='x', labelsize=15)
axes.tick_params(axis='y', labelsize=15)
axes.set_xlabel("value",fontsize=20)
axes.set_ylabel("variable",fontsize=20)
fig.suptitle("Zone Temp variables violinplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper')
#sns.set(font_scale=1)
fig.tight_layout(pad=3)

In [ ]:
#Plot zone temp: 20 vars  
func_hist_plot_rest(vars_series=zoneTemp_vars_series,title="ZoneTemp",ncol=4,colour="lime")
#NOTES: bimodal maybe bc steelgradefamily. pyro2goals are clustered by steelgradefam, so use pyro2goals to make famID and use it as hue with thermocouples

In [ ]:
#Plot zone temp by hue=pyro2Clusters(sort of steelgradefam clusters): 20 vars
func_hist_plot_rest(vars_series=zoneTemp_vars_series,title="ZoneTemp",ncol=4,hue="SteelFamCluster")
#NOTES: bimodal maybe bc steelgradefamily. pyro2goals are clustered by steelgradefam, so use pyro2goals to make famID and use it as hue with thermocouples

In [ ]:
tbl_desc[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]]

### Plot recuperator: 11 vars

In [ ]:
#Plot boxplot recuperator: 11 vars

fig,axes = plt.subplots(5,1,figsize=(20,15))
#sns.boxplot(data=dfXY[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]],orient='h',ax=axes)
sns.boxplot(data=dfXY[recuperator_vars_series[0]],orient='h',ax=axes[0],width=0.5)
axes[0].tick_params(axis='x', labelsize=15)
axes[0].tick_params(axis='y', labelsize=15)
axes[0].set_ylabel(ylabel=recuperator_vars_series[0],rotation='horizontal',horizontalalignment='right',fontsize=15)
axes[0].set_xlabel(xlabel="")

sns.boxplot(data=dfXY[recuperator_vars_series[[1,2]]],orient='h',ax=axes[1],width=0.5)
axes[1].tick_params(axis='x', labelsize=15)
axes[1].tick_params(axis='y', labelsize=15)

sns.boxplot(data=dfXY[recuperator_vars_series[[3,4,5,6]]],orient='h',ax=axes[2],width=0.8)
axes[2].tick_params(axis='x', labelsize=15)
axes[2].tick_params(axis='y', labelsize=15)

sns.boxplot(data=dfXY[recuperator_vars_series[[7,8]]],orient='h',ax=axes[3],width=0.5)
axes[3].tick_params(axis='x', labelsize=15)
axes[3].tick_params(axis='y', labelsize=15)

sns.boxplot(data=dfXY[recuperator_vars_series[[9,10]]],orient='h',ax=axes[4],width=0.5)
axes[4].tick_params(axis='x', labelsize=15)
axes[4].tick_params(axis='y', labelsize=15)

fig.suptitle("Recuperator variables boxplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper',font_scale=1.5)
#sns.set(font_scale=1.5)
fig.tight_layout(pad=3)

In [ ]:
#Plot recuperator: 11 vars
func_hist_plot_rest(vars_series=recuperator_vars_series,title="Recuperator",ncol=2,colour="purple")
#NOTES: bimodal maybe bc steelgradefamily. pyro2goals are clustered by steelgradefam, so use pyro2goals to make famID and use it as hue with thermocouples

In [ ]:
#Plot recuperator: 11 vars
func_hist_plot_rest(vars_series=recuperator_vars_series,title="Recuperator",ncol=2,hue="SteelFamCluster")
#NOTES: bimodal maybe bc steelgradefamily. pyro2goals are clustered by steelgradefam, so use pyro2goals to make famID and use it as hue with thermocouples

In [ ]:
dfXY[recuperator_vars_series].head()#.value_counts()
np.round(dfXY.AP1_FurnaceCombustionAirPressureCV.value_counts().index.values,1)
dfXY[["AP1_FurnaceCombustionAirPressureCV","AP1_FurnaceRWPExitTemp","AP1_FurnaceRWPEntryTemp","AP1_FurnaceCombustionAirPressureSetPoint","AP1_FurnaceDilutionAirTemperatureCV","AP1_FurnaceDilutionAirTemperatureSetPoint"]].apply(pd.Series.value_counts,axis=0)

#We see that those vars have cte and unique values


### Plot pyro: 3 vars

In [ ]:
#Plot boxplot pyro: 3 vars

fig,axes = plt.subplots(figsize=(20,5))
#sns.boxplot(data=dfXY[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]],orient='h',ax=axes)
sns.boxplot(data=dfXY[pyro_vars_series],orient='h',ax=axes,width=0.2)
axes.tick_params(axis='x', labelsize=15)
axes.tick_params(axis='y', labelsize=15)
fig.suptitle("Pyro variables boxplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper')
sns.set(font_scale=1)
fig.tight_layout(pad=3)

In [ ]:
#Plot pyro: 3 vars
ncol=3
nrow=dfXY[pyro_vars_series].shape[1]//ncol+1
var=0

fig,axes = plt.subplots(nrow,ncol,figsize=(25,nrow*20/4))
fig.suptitle("Distribution of numerical pyro variables",fontsize=30)

for r in range(nrow):
    for c in range(ncol):
        if var<len(pyro_vars_series.to_list()):
            hist_container=sns.histplot(data=dfXY,x=pyro_vars_series[var],ax=axes[r,c],kde=True,color="darkorange")
            axes[r,c].set_title(label=pyro_vars_series[var],loc='center',pad=10,fontsize="x-large")
            axes[r,c].set_xlabel(xlabel="",labelpad=10)
            axes[r,c].set_ylabel(ylabel="",labelpad=15)
            axes[r,c].axvline(tbl_desc.loc['cut-',pyro_vars_series[var]], color="red", dashes=(6, 2)) #Cut-
            axes[r,c].axvline(tbl_desc.loc['cut+',pyro_vars_series[var]], color="red", dashes=(6, 2)) #Cut+
            axes[r,c].tick_params(axis='x', labelsize=15)
            axes[r,c].tick_params(axis='y', labelsize=15)

            if pyro_vars_series[var]=="pyro2":
                #Better show graph
                axes[r,c].set_xlim(dfXY.pyro2.min()-50,dfXY.pyro2.max()+50)
                #sns.barplot method will return a list of sub methods, use containers method to access the text label of each bar by passing it through the ax.bar_label function use for loop to iterate through the list of labels and assign each bar to a different label.
                #If using seaborn hist instead of plot.hist, it will add labels to the bins with 0 counts
                #axes[r,c].bar_label(container=hist_container.containers[0])
                barRectangles = axes[r,c].patches
                labels = dfXY.pyro2.value_counts().sort_index().values
                label_i=0
                #Cannot loop through zip(barRectangles,labels) bc barRect is larger as there are rect in the bins with 0 counts
                for rect in barRectangles:#zip()stop when the shortest list ends, iterate through rect bars
                    height = rect.get_height()
                    if height != 0:
                        #print(rect.get_x(),height+6,label)
                        axes[r,c].text(x=rect.get_x()+rect.get_width()/2,y=height+6,s=str(int(height)),ha="center",va="bottom",color="black",fontsize=15)#s=str(labels[label_i])
                        label_i+=1

                #Make room for all labels in plot
                axes[r,c].set_ylim(top=max(labels)+1500)
                #Other way, but not centered horizontally as u dont have the width of the bars
                #for index, value in dfXY.pyro2.value_counts().sort_index().items():
                #    axes[r,c].text(x=index, y=value, s=value, color='yellow', ha='center')

            if c==0:
                axes[r,c].set_ylabel(ylabel="Counts",labelpad=15,fontsize="large")

            var+=1
        #remove unused axes
        #print(len(axes[r,c].get_children()))
        if len(axes[r,c].get_children()) <=10:#empty plots have 10 children
            axes[r,c].set_visible(False)


# for ax in axes.flatten(): #axes.flatten() is a np.array of matplotlib.axes._axes.Axes
#     if not ax.lines: #ax.lines is a matplotlib.axes._base._AxesBase.ArtistList,tells u number of lines in the subplot, if 0 then is empty
#         ax.set_visible(False)

#Keep outer labels
# for ax in fig.get_axes():
#     ax.label_outer()

#Make space between subplots
fig.tight_layout(pad=3)

In [ ]:
dfXY.pyro2.value_counts().sort_index().index#.iloc[:1].values[0]#.index[0]
dfXY.pyro2.value_counts().sort_index().values

In [ ]:
#7 different goals:I think in 3 clusters
dfXY.pyro2.value_counts().sort_values()

### Plot rest vars: 11 vars

In [ ]:
#Plot boxplot rest vars: 11 vars

fig,axes = plt.subplots(11,1,figsize=(20,20))
#sns.boxplot(data=dfXY[zoneTemp_vars_series[~zoneTemp_vars_series.str.contains("CV",regex=False)]],orient='h',ax=axes)
#axes[0].yaxis.set_major_locator(ticker.NullLocator())
sns.boxplot(data=dfXY[rest_vars_series[0]],orient='h',ax=axes[0],width=0.2)
axes[0].tick_params(axis='x', labelsize=15)
axes[0].set_xlabel(rest_vars_series[0], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[1]],orient='h',ax=axes[1],width=0.2)
axes[1].tick_params(axis='x', labelsize=15)
axes[1].set_xlabel(rest_vars_series[1], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[2]],orient='h',ax=axes[2],width=0.2)
axes[2].tick_params(axis='x', labelsize=15)
axes[2].set_xlabel(rest_vars_series[2], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[3]],orient='h',ax=axes[3],width=0.2)
axes[3].tick_params(axis='x', labelsize=15)
axes[3].set_xlabel(rest_vars_series[3], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[4]],orient='h',ax=axes[4],width=0.2)
axes[4].tick_params(axis='x', labelsize=15)
axes[4].set_xlabel(rest_vars_series[4], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[5]],orient='h',ax=axes[5],width=0.2)
axes[5].tick_params(axis='x', labelsize=15)
axes[5].set_xlabel(rest_vars_series[5], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[6]],orient='h',ax=axes[6],width=0.2)
axes[6].tick_params(axis='x', labelsize=15)
axes[6].set_xlabel(rest_vars_series[6], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[7]],orient='h',ax=axes[7],width=0.2)
axes[7].tick_params(axis='x', labelsize=15)
axes[7].set_xlabel(rest_vars_series[7], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[8]],orient='h',ax=axes[8],width=0.2)
axes[8].tick_params(axis='x', labelsize=15)
axes[8].set_xlabel(rest_vars_series[8], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[9]],orient='h',ax=axes[9],width=0.2)
axes[9].tick_params(axis='x', labelsize=15)
axes[9].set_xlabel(rest_vars_series[9], fontsize=15)

sns.boxplot(data=dfXY[rest_vars_series[10]],orient='h',ax=axes[10],width=0.2)
axes[10].tick_params(axis='x', labelsize=15)
axes[10].set_xlabel(rest_vars_series[10], fontsize=15)

fig.suptitle("Rest of variables boxplot",fontsize=30,y=1)
#axes.set_xlim(0,4)
#zone0_fg.map(sns.boxplot)
sns.set_context('paper')
#sns.set(font_scale=1.3)
fig.tight_layout(pad=3)

In [ ]:
#Plot rest vars: 11 vars
func_hist_plot_rest(vars_series=rest_vars_series,title="Rest Furnace",ncol=3,hue="SteelFamCluster",colour="fuchsia")

In [ ]:
#AP1_FurnacePressureSetPoint is cte
dfXY[["AP1_FurnacePressureSetPoint","AP1_FurnacePressureCV"]].apply(pd.Series.value_counts,axis=0)

## Categorical features

In [ ]:
# Values check with .unique()
# 
# Grouped by record counts/bar charts
# 
# Split distribution charts of associated numeric features

In [ ]:
#Categorical vars
categorical_var=dfXY[dfXY.columns[dfXY.dtypes == "category"]].columns.to_list()
categorical_var

In [ ]:
#Barplot for AP_1 vars
nrow=3
ncol=3
var=0
fig, axes = plt.subplots(nrow, ncol,figsize=(15,10))

for r in range(nrow):
    for c in range(ncol):
        if var<len(categorical_var) and var!="grade":
            #plt.subplot_tool()
            #bar=dfXY.loc[:,categorical_var[var]].value_counts().plot(kind="bar",ax=axes[r,c],figsize=(15,10),rot=0,color="green",xlabel='')#,title=var
            counts=dfXY.loc[:,categorical_var[var]].value_counts().values#to make it a list
            values=dfXY.loc[:,categorical_var[var]].value_counts().index.astype(str).values
            bar=axes[r,c].bar(x=values,height=counts,color="green")
            axes[r,c].set_title(label=categorical_var[var],loc='center',pad=10)
            axes[r,c].bar_label(bar,padding=4)
            axes[r,c].set_ylim(top=max(counts)+5000)
            if c==0:
                axes[r,c].set_ylabel(ylabel="counts",labelpad=15)
            if r==2:
                axes[r,c].set_xlabel(xlabel="values",labelpad=5)
            var+=1
#Hide x labels and tick labels for top plots and y ticks for right plots.
#for ax in axes.flat:
#    ax.label_outer()

#Make space between subplots
fig.suptitle("Distribution of categorical variables",fontsize=30)
sns.set(font_scale=1)
fig.tight_layout(pad=3)
#Other way
# set the spacing between subplots
# plt.subplots_adjust(left=0.1,
#                     bottom=0.1, 
#                     right=0.9, 
#                     top=0.9, 
#                     wspace=0.4, 
#                     hspace=0.4)

In [ ]:
#Plot grades
#ax = dfXY.loc[:,"grade"].value_counts().plot(kind="bar",figsize=(20,5),title="GRADES",color="orange",rot=0)
fig,axes = plt.subplots(2,1,figsize=(15,12))
fig.suptitle("Distribution of grades",fontsize=20)
#sns.set_style(style='whitegrid')
#Barplot grades
values = dfXY.loc[:,"grade"].value_counts().index.astype(str).values
counts = dfXY.loc[:,"grade"].value_counts().values
bar_container = axes[0].bar(x=values,height=counts,color="orange")
axes[0].set_title(label="GRADES",pad=15)
axes[0].set_ylabel(ylabel="Counts",labelpad=15)
axes[0].set_xlabel(xlabel="Grade",labelpad=15)
axes[0].bar_label(container=bar_container,fontsize=12)
axes[0].set_ylim(top=max(counts)+400)
#Barplot cumpercentage grades
Percentage = np.round(counts/counts.sum()*100,2)
CumPercentage = np.round(counts.cumsum()/counts.sum()*100,2)
bar_containerPG = axes[1].bar(x=values,height=Percentage,color="green")
#%
axes[1].yaxis.set_major_formatter(PercentFormatter())
axes[1].set_title(label="GRADES CUMULATIVE PERCENTAGE",pad=15)
axes[1].set_ylabel(ylabel="Percentage",labelpad=15)
axes[1].set_xlabel(xlabel="Grade",labelpad=15)
axes[1].bar_label(container=bar_containerPG,fmt='%.2f%%',fontsize=11)
axes[1].set_ylim(top=max(Percentage)+3)
#Cum%
axcum=axes[1].twinx()
axcum.plot(values, CumPercentage, marker = '8', color = 'm',linestyle='solid')
axcum.yaxis.set_major_formatter(PercentFormatter())
axcum.set_ylabel(ylabel="Cumulative Percentage",labelpad=15)

#sns.set(font_scale=1)
fig.tight_layout(pad=3)

In [ ]:
dfXY.loc[:,"grade"].value_counts().values
Percentage
CumPercentage

## Group analysis

In [ ]:
# Correlation plot analysis
# 
# Pairplot analysis (seaborn package has a really good plot for this)

# Observation summary

Record high level comments about what you discovered